## Data Cleaning

### I. Introduction

The purpose of this notebook is to clean the data extracted by the webscrapper in the notebook I.
We clean the strings by removing any irrelevant characters, format the columns to the right type (date, timestamp...).
We remove the rows with inconsistent data (no date, no invitee, etc...)

### II. Cleaning the metadata

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from datetime import datetime, timedelta

In [2]:
jredata = pd.read_csv('jredata.csv') 
jrecontent = pd.read_csv('jrecontent.csv')

In [3]:
jredata=jredata.drop(['Unnamed: 0'], axis=1)
jredata.head()

,date,title,duration
0,2020-06-03,#1485 - Krystal & Saagar,[]
1,2020-06-02,"<a href=""/joe-rogan-experience/1484-reggie-wat...",['2:46:32']
2,2020-05-29,"<a href=""/joe-rogan-experience/1483-jesus-trej...",['2:52:47']
3,2020-05-28,"<a href=""/joe-rogan-experience/1482-jordan-jon...",['2:22:39']
4,2020-05-27,"<a href=""/joe-rogan-experience/jre-mma-show-96...",['2:58:39']


In [4]:
jredata['title'] = jredata['title'].astype(str)
jredata['title'] = jredata['title'].str.replace('<a href="/joe-rogan-experience/','')
jredata['title'] = jredata['title'].str.replace('index.html">','')
jredata['title'] = jredata['title'].str.replace('</a>','')
jredata['title'] = jredata['title'].str.replace('PODCAST','')
jredata['title'] = jredata['title'].str.replace('#','')
jredata['title'] = jredata['title'].str.replace('\d+','')
jredata['title'] = jredata['title'].str.replace('(.*)-[^-]*(.*)-','')
jredata['title'] = jredata['title'].str.replace('-','')
jredata.head()

,date,title,duration
0,2020-06-03,Krystal & Saagar,[]
1,2020-06-02,Reggie Watts,['2:46:32']
2,2020-05-29,Jesus Trejo,['2:52:47']
3,2020-05-28,Jordan Jonas,['2:22:39']
4,2020-05-27,wittman/JRE MMA Show with Justin Gaethje &amp...,['2:58:39']


In [5]:
jredata.rename(columns={"title": "guest"}, inplace=True)

In [6]:
jredata[jredata['guest'].str.contains("May")]

,date,guest,duration
11,2020-05-17,"May ,",['4:04:17']
189,2019-07-29,Maynard James Keenan,['1:52:37']
442,2018-05-27,"May ,",['3:15:51']
595,2017-08-28,schaub/Mayweather vs. McGregor Recap with Bren...,['3:10:11']
600,2017-08-16,Mayweather vs. McGregor,['2:55:38']
615,2017-07-12,Maynard James Keenan,['3:02:01']
637,2017-05-28,"May ,",['3:30:24']
816,2016-05-30,"May ,",['4:31:40']
1140,2014-06-01,"May ,",['2:56:43']
1151,2014-05-11,"May ,",['2:57:01']


In [7]:
jredata=jredata[~jredata.guest.str.contains("January ,")]
jredata=jredata[~jredata.guest.str.contains("February ,")]
jredata=jredata[~jredata.guest.str.contains("Feb. ,")]
jredata=jredata[~jredata.guest.str.contains("March ,")]
jredata=jredata[~jredata.guest.str.contains("April ,")]
jredata=jredata[~jredata.guest.str.contains("May ,")]
jredata=jredata[~jredata.guest.str.contains("June ,")]
jredata=jredata[~jredata.guest.str.contains("July ,")]
jredata=jredata[~jredata.guest.str.contains("August ,")]
jredata=jredata[~jredata.guest.str.contains("September ,")]
jredata=jredata[~jredata.guest.str.contains("Sept. ,")]
jredata=jredata[~jredata.guest.str.contains("October ,")]
jredata=jredata[~jredata.guest.str.contains("November ,")]
jredata=jredata[~jredata.guest.str.contains("December ,")]

In [8]:
jredata[jredata['guest'].str.contains("Sept")]

,date,guest,duration


In [9]:
jredata['duration'] = jredata['duration'].astype(str)
jredata['duration'] = jredata['duration'].str.replace('[','')
jredata['duration'] = jredata['duration'].str.replace(']','')
jredata['duration'] = jredata['duration'].str.replace("'",'')


### III. Cleaning the text

In [10]:
jrecontent.head()

,Unnamed: 0,date,title,duration,text
0,0,2020-06-03,#1485 - Krystal & Saagar,[],"['Hey friends, how you doing this episode, the..."
1,1,2020-06-02,"<a href=""/joe-rogan-experience/1484-reggie-wat...",['2:46:32'],"['Hello, friends, welcome to the show this epi..."
2,2,2020-05-29,"<a href=""/joe-rogan-experience/1483-jesus-trej...",['2:52:47'],"['Pillow friends, welcome to the show this epi..."
3,3,2020-05-28,"<a href=""/joe-rogan-experience/1482-jordan-jon...",['2:22:39'],"['Hello, friends, welcome to the show this epi..."
4,4,2020-05-27,"<a href=""/joe-rogan-experience/jre-mma-show-96...",['2:58:39'],"['Oh hi, friends welcome to the show this absu..."


In [11]:
jrecontent['title'] = jrecontent['title'].astype(str)
jrecontent['title'] = jrecontent['title'].str.replace('<a href="/joe-rogan-experience/','')
jrecontent['title'] = jrecontent['title'].str.replace('index.html">','')
jrecontent['title'] = jrecontent['title'].str.replace('</a>','')
jrecontent['title'] = jrecontent['title'].str.replace('PODCAST','')
jrecontent['title'] = jrecontent['title'].str.replace('#','')
jrecontent['title'] = jrecontent['title'].str.replace('\d+','')
jrecontent['title'] = jrecontent['title'].str.replace('(.*)-[^-]*(.*)-','')
jrecontent['title'] = jrecontent['title'].str.replace('-','')
jrecontent.rename(columns={"title": "guest"}, inplace=True)
jrecontent['duration'] = jrecontent['duration'].astype(str)
jrecontent['duration'] = jrecontent['duration'].str.replace('[','')
jrecontent['duration'] = jrecontent['duration'].str.replace(']','')
jrecontent['duration'] = jrecontent['duration'].str.replace("'",'')
jrecontent['text'] = jrecontent['text'].str.replace('-','')
jrecontent['text'] = jrecontent['text'].str.replace('\d+','')
jrecontent['text'] = jrecontent['text'].str.replace('[','')
jrecontent['text'] = jrecontent['text'].str.replace(']','')
jrecontent['text'] = jrecontent['text'].str.replace("'",'')
jrecontent['text'] = jrecontent['text'].str.lower() 
jrecontent=jrecontent[~jrecontent.guest.str.contains("January ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("February ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("Feb. ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("March ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("April ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("May ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("June ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("July ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("August ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("September ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("Sept. ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("October ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("November ,")]
jrecontent=jrecontent[~jrecontent.guest.str.contains("December ,")]
jrecontent.head()

,Unnamed: 0,date,guest,duration,text
0,0,2020-06-03,Krystal & Saagar,,"hey friends, how you doing this episode, the p..."
1,1,2020-06-02,Reggie Watts,2:46:32,"hello, friends, welcome to the show this episo..."
2,2,2020-05-29,Jesus Trejo,2:52:47,"pillow friends, welcome to the show this episo..."
3,3,2020-05-28,Jordan Jonas,2:22:39,"hello, friends, welcome to the show this episo..."
4,4,2020-05-27,wittman/JRE MMA Show with Justin Gaethje &amp...,2:58:39,"oh hi, friends welcome to the show this absurd..."


### IV. Formating the date and timestamp column

In [12]:
jredata['date'] = pd.DatetimeIndex(jredata['date']).date
jrecontent['date'] = pd.DatetimeIndex(jrecontent['date']).date

In [13]:
jredata.head()

,date,guest,duration
0,2020-06-03,Krystal & Saagar,
1,2020-06-02,Reggie Watts,2:46:32
2,2020-05-29,Jesus Trejo,2:52:47
3,2020-05-28,Jordan Jonas,2:22:39
4,2020-05-27,wittman/JRE MMA Show with Justin Gaethje &amp...,2:58:39


### V. removing inconsistent data

In [14]:
jredata = jredata[jredata.duration != 'NaT']
jrecontent = jrecontent[jredata.duration != 'NaT']

C:\Users\benja\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [15]:
print(jredata.shape)
print(jrecontent.shape)

(1608, 3)
(1056, 5)


In [16]:
jredata.dropna(subset=['duration'], inplace=True)
jrecontent.dropna(subset=['duration'], inplace=True)

In [17]:
print(jredata.shape)
print(jrecontent.shape)

(1608, 3)
(1056, 5)


In [21]:
jredata.to_csv('jredataclean.csv')
jrecontent.to_csv('jrecontentclean.csv')